# Experience 5

**Name**:  Tina Yung-Fang Tu
**Time Spent**: 10hr

In [29]:
import numpy as np
import pandas as pd
import itertools as it
from scipy.stats import chi2

In [6]:
# please set the path to your data directory here
path = "./EPI511/"

# please use the following function (or something like it) to read files
def pname(name):
    '''Prepend the path to the filename'''
    return path + '/' + name

def popen(name):
    '''Open file in the path'''
    return open(pname(name))

In [7]:
#################### functions to read in data ##################
def read_snp(file):
    '''Read a snp file into a pandas dataframe'''
    return(pd.read_table(
        file,
        sep='\s+', # columns are separated by whitespace
        # names of the columns
        names=[None, 'chromosome', 'morgans', 'position', 'ref', 'alt'],
        index_col=0))

SNPs = read_snp(path + 'HapMap3.snp') 

def get_chr_range(chromosome):
    '''Returns the range of positions where SNPs for a chromosome are kept'''
    filt = SNPs.query('chromosome=={}'.format(chromosome))
    start = SNPs.index.get_loc(filt.iloc[0].name)
    stop  = SNPs.index.get_loc(filt.iloc[-1].name) + 1
    return(start, stop)

def read_geno(file):
    '''Reads a geno file into a masked numpy matrix'''
    return(np.genfromtxt(
        file,               # the file
        dtype='uint8',      # read the data in as 1-byte integers
        delimiter=1,        # 1-byte width data
        missing_values=9,   # 9 indicates missing data
        usemask=True        # return a masked array
    ))

def read_geno_pop_chr(pop, chromosome):
    '''Reads a slice of a geno file into a masked numpy matrix'''
    f = open(path + pop + '.geno')      # open the file
    (start, stop) = get_chr_range(chromosome)
    s = it.islice(f, start, stop) # slice the file only keeping SNPs of chr
    return read_geno(s)

## (1) Polygenic prediction

Consider a set of 1,000 unlinked SNPs in the 112 CEU individuals defined by the first SNP and every 125th SNP thereafter (SNPs 0, 50, …, 124875, if the first SNP is SNP 0).  Assign the first 100 SNPs as causal SNPs and other 900 SNPs as non-causal SNPs. Simulate quantitative phenotypes for CEU individuals by assuming that causal SNPs have effect size per normalized genotype = 0.1 (note that this is different from effect size per allele) and null SNPs have eff75ect size per normalized genotype = 0.  Label this CEU data as “training data”.  Use the same effect sizes to simulate “test data” consisting of real genotypes and simulated phenotypes in 88 TSI individuals.  Implement a polygenic prediction scheme using all 1,000 SNPs in which you use training data to estimate ATT effect sizes (i.e. correlations between genotypes and phenotypes) and then use estimated effect sizes to build predicted phenotypes in test data.    (TSI test data genotypes can be normalized using allele frequencies from CEU training data.) (Note that because this is a simulation, the true effect sizes are known.  However, the true effect sizes should not be used to build predicted phenotypes).  What is the prediction r2 of the predicted phenotypes (vs. true phenotypes) in the test data?  Is the prediction r2 significantly different from 0?  Does the prediction r2 agree with the derivation provided in Week 9 slides?

### Solution

In [8]:
# Reads a slice of a geno file into a numpy matrix
def read_geno_pop_islice(pop, *args):
    f = popen(pop + '.geno') 
    s = it.islice(f, *args)  
    return read_geno(s)

# Calculate allele frequency
def calculate_af(geno):
    return np.ma.mean(geno, axis=1).filled(-1) / 2

# Normalize geno matrix
def normalize_geno(geno, p=None):
    if p is None: p = calculate_af(geno)
    return ( (geno - (2 * p)[:, np.newaxis]) / np.sqrt(2 * p * (1 - p))[:, np.newaxis]).filled(0)

def compute_r2(snp1, snp2):
    # Use masks to filter valid data
    valid = ~snp1.mask & ~snp2.mask
    if np.sum(valid) < 2:
        return np.nan

    gA = snp1[valid].astype(float)
    gB = snp2[valid].astype(float)
    
    if np.std(gA) == 0 or np.std(gB) == 0:
        return np.nan
    
    # Genotype count for allele
    gA_mean = np.mean(gA) 
    gB_mean = np.mean(gB) 

    Na = len(gA)
    Nb = len(gB)

    var_a = ((gA - gA_mean)**2).sum()/Na
    var_b = ((gB - gB_mean)**2).sum()/Nb

    # Compute r^2
    numerator = ((gA *gB).mean() - (gA_mean * gB_mean))**2
    denominator = var_a * var_b
    if denominator == 0:
        return np.nan
    return numerator / denominator

def compute_pearsonr(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)

    x_centered = x - x_mean
    y_centered = y - y_mean

    numerator = np.sum(x_centered * y_centered)
    denominator = np.sqrt(np.sum(x_centered ** 2)) * np.sqrt(np.sum(y_centered ** 2))

    if denominator == 0:
        return 0  
    return numerator / denominator

In [44]:
CEU_geno = read_geno_pop_islice('CEU', 0, 125000, 125)
CEU_af = calculate_af(CEU_geno)
X_CEU = normalize_geno(CEU_geno, CEU_af)

# Define effect sizes and phenotype
beta = np.zeros(1000)
beta[:100] = 0.1
Y_CEU = X_CEU.T.dot(beta)

# Estimate effect sizes using correlation
r_values = np.zeros(X_CEU.shape[0])  # 1000 SNPs
for i in range(X_CEU.shape[0]):
    r_values[i] = compute_pearsonr(X_CEU[i], Y_CEU)

# Load and normalize TSI genotypes using CEU allele frequencies
TSI_geno = read_geno_pop_islice('TSI', 0, 125000, 125)
X_TSI = np.zeros(TSI_geno.shape, dtype=float)

for i in range(1000):
    X_TSI[i] = np.ma.filled((TSI_geno[i] - 2 * CEU_af[i]) / np.sqrt(2 * CEU_af[i] * (1 - CEU_af[i])), fill_value=0)

# Simulate true TSI phenotypes using true beta
Y_TSI_true = np.dot(beta, X_TSI)

# Predict TSI phenotypes using estimated r-values
Y_TSI_pred = np.dot(r_values, X_TSI)

corr = compute_pearsonr(Y_TSI_true, Y_TSI_pred)
chisq = Y_TSI_true.shape[0] * corr**2
p_value = chi2.sf(chisq, df=1)
print(f"Prediction correlation (R): {corr}")
print(f"Prediction R²: {corr**2}")

print(f"Chi-square: {Y_TSI_true.shape[0] * corr**2}")
print(f"p-value: {p_value}")

Prediction correlation (R): 0.34936079770037815
Prediction R²: 0.12205296696984454
Chi-square: 10.740661093346318
p-value: 0.0010480688631516852


*comments*

The  r2 of the predicted phenotypes (vs. true phenotypes) in the test data is 0.122, and the calculated chi-sq statistic is around 10.74. Therefore, the prediction r2 significantly different from 0 (p-value = 0.001). According to theoretical derivations, the r2 value should be around  (h_g^2 × h_g^2) / (h_g^2 + M/N). Here, h_g^2 = 1, M = 1000 and N = 112. Hence, the theoretical value of r2 is 0.10072. It aligns closely with our derivation, and the slight difference might be due to randomness or the presence of masked values (missing data) in the genome matrix. 

## (2) Modified prediction

Using the simulated CEU training data and TSI test data from (1), implement a polygenic prediction scheme using a P-value threshold, in which only markers with P-values beneath a threshold (for ATT χ2 association test) are included in the predictions.  (TSI test data genotypes can be normalized using allele frequencies from CEU training data.)  Implement this scheme for various choices of P-value thresholds.  How do the results vary?  Discuss. 
Hint: below are $\chi^2$ thresholds $\chi^2_{THRESH}$ corresponding to various $P$-value thresholds $P_{THRESH}$:

| $P_{THRESH}$ | 1.0 | 0.5 | 0.2 | 0.1 | 0.05 | 0.02 | 0.01 |
|---|---|---|---|---|---|---|---|
| $\chi^2_{THRESH}$ | 0.000 | 0.455 | 1.642 | 2.706 | 3.852 | 5.412 | 6.635 |

(You can also just use scipy.stats.chi2.)

### Solution

In [60]:
# Armitage trend test
def armitage_trend_test(genotype, phenotype):
    if np.ma.isMaskedArray(genotype):
        valid = ~genotype.mask
    else:
        valid = np.ones_like(genotype, dtype=bool)

    valid_genotype = genotype[valid]
    valid_phenotype = phenotype[valid] 

    if len(valid_genotype) == 0:
        return np.nan

    r = np.corrcoef(valid_genotype, valid_phenotype)[0, 1]
    if np.isnan(r):
        return np.nan

    N = len(valid_genotype)
    return N * r**2

In [61]:
att_chisq = np.zeros(1000)
for i in range(1000):
    att_chisq[i] = armitage_trend_test(CEU_geno[i], Y_CEU)

# Set threshold
chi2_thresholds = [0.000, 0.455, 1.642, 2.706, 3.842, 5.412, 6.635]
results = []
N = len(Y_TSI_true)

for chi2_thresh in chi2_thresholds:
    selected = att_chisq > chi2_thresh
    num_sig = np.sum(selected)
    num_causal = np.sum(selected[:100])

    if num_sig == 0:
        results.append((chi2_thresh, 0, 1.0, 0, 0))  
        continue

    weights = np.zeros_like(r_values)
    weights[selected] = r_values[selected]

    Y_TSI_pred_sig = np.dot(weights, X_TSI)
    corr_sig = compute_pearsonr(Y_TSI_true, Y_TSI_pred_sig)
    r2_sig = corr_sig ** 2
    chisq_stat = N * r2_sig

    results.append((chi2_thresh, r2_sig, num_sig, num_causal))

print("χ² Threshold\tR²\t\tNum SNPs\tCausal SNPs")
for chi2_t, r2, num, causal in results:
    print(f"{chi2_t:.3f}\t\t{r2:.4f}\t\t{num}\t\t{causal}")

χ² Threshold	R²		Num SNPs	Causal SNPs
0.000		0.1221		1000		100
0.455		0.1525		507		72
1.642		0.1054		229		41
2.706		0.0992		127		26
3.842		0.0583		65		13
5.412		0.0828		25		6
6.635		0.0365		12		5


*comments*

Based on the results, we can see that when using all SNPs (χ² > 0.0), R² is 0.1221. However, this includes many non-causal SNPs (900 out of 1000), which likely dilutes the predictive signal. When 507 SNPs (72 causal) are included, we achieve the highest R² among these threshold values. In other words, this threshold captures useful signal without adding too much noise. As we increase the threshold, we keep fewer SNPs. Although they are more strongly associated, we lose too many causal ones, so R² drops. Here, the optimal threshold appears to be around χ² > 0.455 (p = 0.5), where predictive accuracy is maximized without filtering out too many relevant SNPs.

## (3) BLUP

Using the simulated CEU training data and TSI test data from (1), compute BLUP predictions for TSI test samples. Does the prediction r2 agree with theoretical derivations?
(Note: if necessary, you may assume that the total genetic variance is 0.99 and the environmental variance is 0.01 to avoid problems inverting the phenotypic covariance matrix.)
(Note: it will be necessary to invert the genetic relationship matrix in order to compute BLUP predictions.


### Solution

In [46]:
M = X_CEU.shape[0]
sigma_g2 = 0.99
sigma_e2 = 0.01

# GRM
A = X_CEU.T @ X_CEU / M
A_star = X_TSI.T @ X_CEU / M
V = sigma_g2 * A + sigma_e2 * np.eye(A.shape[0])
V_inv = np.linalg.inv(V)

Y_TSI_blup = sigma_g2 * A_star @ V_inv @ Y_CEU

# Evaluate
corr_blup = compute_pearsonr(Y_TSI_true, Y_TSI_blup)
print(f"BLUP Prediction R²: {corr_blup**2}")

BLUP Prediction R²: 0.1705653246696614


*comments*

According to theoretical derivations, 
r2 = (h_g^2 × h_g^2) / [h_g^2 + (1 - r^2) × M/N]. Here, h_g^2 = 1, M = 1000 and N = 112. 

Hence, the theoretical value of BLUP Prediction R² is 0.112. Our results are close to theoretical derivations, but do not match exactly, which might be due to random variation and the presence of masked values (missing data). 

## (4)  LDpred

(a) Using the simulated CEU training data and TSI test data from (1), implement a polygenic prediction scheme using the LDpred method in the special case of no LD between SNPs, for various values of the proportion of causal SNPs.  Does the correct value of the proportion of causal SNPs produce the highest prediction r2? (b) Repeat the simulation using effect sizes for causal SNPs sampled from a normal distribution with standard deviation 0.1 (variance 0.01).  Does the correct value of the proportion of causal SNPs produce the highest prediction r2?
(Hint: you can sample values from the standard normal distribution N(0,1) by rejection sampling: sample uniformly from [−10,10] (a standard normal variable is unlikely to be outside this range) and retain the sampled value with probability proportional to the standard normal density.)


*Note the statement from the assignment: "do not use a built-in normally distributed random number generator in problem (4)."*

### Solution

In [52]:
# Define parameters
h_g2 = 1.0  
M = 1000
N = X_CEU.shape[1]
proportion = [1, 0.5, 0.1, 0.05, 0.01]

ldpred_results = []

for p in proportion:
    v1 = h_g2 / (M * p) + 1 / N
    v0 = 1 / N

    sqrt_v1 = np.sqrt(v1)
    sqrt_v0 = np.sqrt(v0)

    exp1 = np.exp(-r_values**2 / (2 * v1))
    exp0 = np.exp(-r_values**2 / (2 * v0))

    numerator = (p / sqrt_v1) * exp1
    denominator = numerator + ((1 - p) / sqrt_v0) * exp0
    posterior_prob = numerator / denominator 

    coeff = h_g2 / (h_g2 + (M * p / N))
    ldpred_betas = coeff * posterior_prob * r_values

    # Predict in TSI
    Y_pred = np.dot(ldpred_betas, X_TSI)
    r = compute_pearsonr(Y_TSI_true, Y_pred)
    r2 = r ** 2
    ldpred_results.append((p, r2))

print("Proportion causal SNPs (p)\tLDpred R²")
for p, r2 in ldpred_results:
    print(f"{p:<25}\t{r2}")

Proportion causal SNPs (p)	LDpred R²
1                        	0.12205296696984451
0.5                      	0.12193676712119224
0.1                      	0.07987983532121183
0.05                     	0.049392713787376176
0.01                     	0.005951517293815547


In [53]:
# Generate random effect sizes
def sample_normal(n, std=0.1):
    samples = []
    while len(samples) < n:
        x = np.random.uniform(-10, 10)
        accept_prob = np.exp(-x**2 / 2)  # standard normal PDF (unnormalized)
        if np.random.uniform(0, 1) < accept_prob:
            samples.append(x * std)
    return np.array(samples)

M = 1000
causal_indices = np.zeros(M, dtype=bool)
causal_indices[:100] = True
random_effect = np.zeros(M)
random_effect[causal_indices] = sample_normal(np.sum(causal_indices))

Y_CEU_normal = np.dot(random_effect, X_CEU)
Y_TSI_true_normal = np.dot(random_effect, X_TSI)

r_values_normal = np.zeros(M)
for i in range(M):
    r_values_normal[i] = compute_pearsonr(X_CEU[i], Y_CEU_normal)

# Rerun LDpred with normal effect sizes
ldpred_results_normal = []
for p in proportion:
    v1 = h_g2 / (M * p) + 1 / N
    v0 = 1 / N

    sqrt_v1 = np.sqrt(v1)
    sqrt_v0 = np.sqrt(v0)

    exp1 = np.exp(-r_values_normal**2 / (2 * v1))
    exp0 = np.exp(-r_values_normal**2 / (2 * v0))

    numerator = (p / sqrt_v1) * exp1
    denominator = numerator + ((1 - p) / sqrt_v0) * exp0
    posterior_prob = numerator / denominator

    coeff = h_g2 / (h_g2 + (M * p / N))
    ldpred_betas = coeff * posterior_prob * r_values_normal

    Y_pred = np.dot(ldpred_betas, X_TSI)
    r = compute_pearsonr(Y_TSI_true_normal, Y_pred)
    r2 = r ** 2
    ldpred_results_normal.append((p, r2))

# Print results
print("Proportion causal SNPs (p)\tLDpred R² (Normal Effects)")
for p, r2 in ldpred_results_normal:
    print(f"{p:<25}\t{r2}")

Proportion causal SNPs (p)	LDpred R² (Normal Effects)
1                        	0.028411494958547404
0.5                      	0.04171619339597841
0.1                      	0.10390632713954469
0.05                     	0.10353257559152029
0.01                     	0.029285667123192027


*comments*

From the first results above, we observe that even though the true proportion of causal SNPs is 0.1, the highest r2 is achieved when p = 1 or 0.5. This is because in LDpred, we assume that causal SNPs have effect sizes drawn from a normal distribution. However, in this stimulation, we have exactly 100 causal SNPs with constant effect size = 0.1. Because LDpred assumes normal effect sizes, its posterior formula expects a mixture of small and large effects, but our actual effects are all moderate and equal. So when we set p = 0.1, LDpred expects a few large effects and shrinks others hard. But in truth, all 100 causal SNPs have the same size, so even smaller marginal effects are meaningful, and LDpred underestimates the importance of many of them. Higher p (0.5 or 1 in this case) leads to less shrinkage overall, preserving more of the signal. 

In the second part, the highest r2 (0.1039) is achieved when p = 0.1,  which matches the true proportion of causal SNPs in our simulation. This is consistent with the fact that LDpred performs best when its prior on the proportion of causal variants aligns with reality. When p > 0.1, LDpred under-regularizes, meaning that it includes too many non-causal SNPs. When p < 0.1, LDpred over-shrinks the effect sizes and loses real signal. In short, we see that when SNP effect sizes are truly normally distributed, LDpred achieves optimal prediction accuracy at the correct causal proportion p = 0.1. 

## (5) MLMi mixed model association $\chi^2$  statistics

Consider a set of 1,000 unlinked SNPs in the 112 CEU individuals consisting of 100 unlinked SNPs on each of chromosomes 1-10 defined by the first SNP and every 125th SNP thereafter (SNPs 0, 50, …, 12375, if the first SNP is SNP 0) on each of those chromosomes.  Assign the first SNP on each chromosome as a causal SNP (10 causal SNPs total) and the remaining 990 SNPs as non-causal SNPs.  Simulate quantitative phenotypes for CEU individuals by assuming that each causal SNP has effect size per normalized genotype = sqrt(1/10) = 0.316.  Compute MLMi mixed model association χ2 statistics as described in Week 10 slides (EMMAX method).  For simplicity, you can assume σg2 = 1 and σe2 = 0.  How do average MLMi mixed model association χ2 statistics at all, null, and causal SNPs compare to average ATT χ2 association statistics at the same SNPs?  Does this agree with theoretical derivations?  (Note: it will be necessary to invert the genetic relationship matrix in order to compute MLMi mixed model association χ2 statistics.  Built-in matrix inversion routines will be provided. Also, it is fine to use numpy.linalg.inv() to compute the inverse of a matrix.)

### Solution

In [55]:
def read_geno_pop_chr_slice(pop, chromosome, end, step):
    '''Reads a slice of a geno file into a numpy matrix'''
    f = popen(pop + '.geno')      # open the file
    (start_chr, stop_chr) = get_chr_range(chromosome)
    stop = start_chr+end
    s = it.islice(f, start_chr,stop,step) # slice the file
    return read_geno(s)

In [80]:
CEU_genos = [read_geno_pop_chr_slice('CEU', chromosome, 12500 ,125) for chromosome in range(1,11)]
CEU_geno_whole  = np.ma.vstack(CEU_genos)
CEU_whole_norm = normalize_geno(CEU_geno_whole)

# Assign effect sizes
causal_index_new = np.arange(0, M, 100)
effect_sizes = np.zeros(M)
effect_sizes[causal_index_new] = 0.316

# Simulate phenotype
Y_CEU_new = CEU_whole_norm.T.dot(effect_sizes)

# Build GRM
M = CEU_whole_norm.shape[0]
A_new = CEU_whole_norm.T.dot(CEU_whole_norm) / M 

V_new = sigma_g2 * A_new + sigma_e2 * np.eye(A_new.shape[0])

# Compute MLMi statistics
MLMi = (CEU_whole_norm.dot(np.linalg.inv(V_new).dot(Y_CEU_new)))**2/CEU_whole_norm.dot(np.linalg.inv(V_new).dot(CEU_whole_norm.T))
MLMi_diag = np.diagonal(MLMi)

causal_mask = np.zeros(1000, dtype=bool)
causal_mask[causal_index_new] = True

MLMi_chi2_causal = np.mean(MLMi_diag[causal_mask])
MLMi_chi2_null = np.mean(MLMi_diag[~causal_mask])
MLMi_chi2_all = np.mean(MLMi_diag)

print('-------- MLMi Chi-sq	----------')
print("Average χ² for causal SNPs:", MLMi_chi2_causal)
print("Average χ² for null SNPs:", MLMi_chi2_null)
print("Average χ² for all SNPs:", MLMi_chi2_all)

# Compute ATT statistics using the normalized genotype matrix
att_chi2_stats = np.zeros(1000)
for i in range(1000):
    att_chi2_stats[i] = armitage_trend_test(CEU_whole_norm[i, :], Y_CEU_new) 

avg_chi2_causal = np.mean(att_chi2_stats[causal_mask])
avg_chi2_null = np.mean(att_chi2_stats[~causal_mask])
avg_chi2_all = np.mean(att_chi2_stats)

print('-------- ATT Chi-sq	----------')
print("Average χ² for causal SNPs:", avg_chi2_causal)
print("Average χ² for null SNPs:", avg_chi2_null)
print("Average χ² for all SNPs:", avg_chi2_all)

-------- MLMi Chi-sq	----------
Average χ² for causal SNPs: 8.037746018354785
Average χ² for null SNPs: 0.6983087090873453
Average χ² for all SNPs: 0.7717030821800197
-------- ATT Chi-sq	----------
Average χ² for causal SNPs: 10.079384931376692
Average χ² for null SNPs: 0.9752888281713061
Average χ² for all SNPs: 1.0663297892033599


*comments*
The theoretical value of MLMi chi-square statistics:
1. MLMi causal: 

Since E[chi_sq for all SNPs] = P(causal) × E[chi_sq for causal SNPs] + P(null) × E[chi_sq for null SNPs], we can plug in the numbers. 
1 = 0.01 × E[chi_sq for causal SNPs] + 0.99 × 0.888. 
E[chi_sq for causal SNPs] = 12.088

2. MLMi null: (1 - r2 × h_g^2) / [h_g^2 × N/ M + (1 - r2 × h_g^2)]. Here, r2 ~ 0.112 as previously derived, so MLMi null ~ 0.888
3. MLMi all: 1

The theoretical value of ATT chi-square statistics:
1. ATT causal: 1 + (h_g^2 × N/ M_causal) = 1 + 1 × 112/10 = 12.2
2. ATT null: 1
3. ATT all: 1 + (h_g^2 × N/ M) = 1 + 1 × 112/1000 = 1.112

Based on the results above, we observe that MLMi tends to deflate average test statistics, making it more conservative. This is especially evident for null SNPs, which are corrected too aggressively, and for causal SNPs, whose chi-sq values fall slightly below the theoretical expectation. This deflation arises because MLMi uses a GRM that includes causal SNPs, absorbing part of their signal and leading to a slight loss of power.

In contrast, ATT results show mild inflation, with average χ² values slightly above 1, which is consistent with expectations under a polygenic model. While the average χ² for causal SNPs is slightly below the theoretical value of 12.2, it remains close, indicating that ATT retains more of the causal signal.

In conclusion, MLMi offers better control of false positives but can reduce power due to overcorrection, particularly when causal variants are included in the GRM. ATT retains higher power but is more susceptible to inflation from polygenic background effects.